In [569]:
# !pip install plotly panel

In [570]:
import os 
import json
import pandas as pd 
import traceback
import re

# Plotting 
import plotly.express as px
import panel as pn

from dotenv import load_dotenv
load_dotenv()

# LangChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
from langchain_community.document_loaders.csv_loader import CSVLoader

import warnings
warnings.filterwarnings("ignore")

In [571]:
KEY = os.getenv("OPENAI_API_KEY")

In [572]:
llm = ChatOpenAI(openai_api_key = KEY, model_name = "gpt-3.5-turbo", temperature = 0.5)

### Read transaction df

In [573]:
# Read the transaction file
df = pd.read_csv("./Copy of My-Activity.csv", header = None)
df = df.replace("â‚¹", "", regex=True)
df.head(10)

,0
0,Google Pay
1,Paid 492.00 using Bank Account XXXXXX5367
2,"24 Feb 2024, 23:42:42 IST"
3,Products:
4,Google Pay
5,Details:
6,BrRLz7GeTu+gMU1Y
7,Completed
8,Google Pay
9,Paid 477.22 to Zomato using Bank Account XXXXX...


In [574]:
# Initialize an empty list to collect the transaction df
activity_df_list = []

# Temporary dictionary to store the current transaction"s df
temp_df = {"Paid Info": None, "Date": None, "Time": None, 
            "Product": None, "Details": None, "Status": None}

# Flag to mark when we are capturing multi-line details
capturing_details = False

for index, row in df.iterrows():
    text = row[0]

    # if "Paid" in text or "Received" in text or "Sent" in text:
    if "Paid" in text:
        # if "Zomato" in text or "Spotify" in text or  "Netflix" in text or "StockEdge" in text or "Udemy" in text or "BOOKMYSHOW" in text or "AMRIT ENTERPRISES" in text or "MUPPARAPU LAVAN" in text:
            temp_df["Paid Info"] = text
    elif "IST" in text:
        temp_df["Date"] = text.split(",")[0]
        temp_df["Time"] = text.split(",")[1]
    elif text.startswith("Products:"):
        # We expect the next line to be "Google Pay" or similar product name
        continue
    elif text.startswith("Google Pay") and not temp_df["Product"]:
        temp_df["Product"] = text
    elif "Details:" in text:
        # We expect the next line to contain the details
        capturing_details = True
    elif capturing_details:
        temp_df["Details"] = text
        capturing_details = False
    elif text.startswith("Completed"):
        temp_df["Status"] = text
        # Add the transaction to the list and reset temp_df
        activity_df_list.append(temp_df.copy())
        temp_df = {key: None for key in temp_df}

# Convert the list of transaction df to a dfFrame
activity_df = pd.DataFrame(activity_df_list)
activity_df.dropna(subset=["Paid Info"], inplace = True)
activity_df

,Paid Info,Date,Time,Product,Details,Status
0,Paid 492.00 using Bank Account XXXXXX5367,24 Feb 2024,23:42:42 IST,Google Pay,BrRLz7GeTu+gMU1Y,Completed
1,Paid 477.22 to Zomato using Bank Account XXXXX...,24 Feb 2024,23:29:05 IST,Google Pay,Kbuxp7NqSvmNjKCA,Completed
3,Paid 644.00 using Bank Account XXXXXX5367,24 Feb 2024,16:49:14 IST,Google Pay,lbcbpAmpT2ief1Fx,Completed
4,Paid 140.00 to HOTEL SRI RAGHAVENDRA using Ban...,24 Feb 2024,12:12:53 IST,Google Pay,sO+iBrMVRb6AYIHm,Completed
7,Paid 40.00 to MUPPARAPU LAVAN using Bank Accou...,23 Feb 2024,21:05:45 IST,Google Pay,d4DinRkqSRWqjeCh,Completed
...,...,...,...,...,...,...
553,Paid 15.00 to Groww using Bank Account XXXXXX2322,22 Sept 2021,15:11:05 IST,Google Pay,KC1AT7PCT+uKjP0h,Completed
554,Paid 20.00 to Groww using Bank Account XXXXXX2322,22 Sept 2021,15:01:11 IST,Google Pay,As530AmcTsSGVOM3,Completed
555,Paid 236.00 to Groww using Bank Account XXXXXX...,22 Sept 2021,15:00:05 IST,Google Pay,MBj1yQkeSmSCLhJa,Completed
557,"Paid 10,000.00 to Groww using Bank Account XXX...",2 Sept 2021,19:12:57 IST,Google Pay,SdquwSVQRaK3vVX/,Completed


In [575]:
# activity_df.to_csv("Activity_df(New).csv")

In [576]:
# Function to clean and standardize date strings
def clean_date(date_str):
    # Remove non-date characters (like "\x0c")
    cleaned_str = re.sub(r"[^\w\s]", "", date_str)

    # Standardize month abbreviations
    month_replacements = {
        "Jan": "Jan", "Feb": "Feb", "Mar": "Mar", "Apr": "Apr", "May": "May", "Jun": "Jun",
        "Jul": "Jul", "Aug": "Aug", "Sep": "Sep", "Sept": "Sep", "Oct": "Oct", "Nov": "Nov", "Dec": "Dec"
    }
    for k, v in month_replacements.items():
        cleaned_str = cleaned_str.replace(k, v)

    return cleaned_str

# Apply the cleaning function to the "Date" column
activity_df["Cleaned Date"] = activity_df["Date"].apply(clean_date)

# Attempt conversion again
activity_df["Converted Date"] = pd.to_datetime(activity_df["Cleaned Date"], format="%d %b %Y", errors="coerce").dt.strftime("%Y-%m-%d")

# Display the first few rows of the modified dfframe to verify changes
activity_df[["Date", "Cleaned Date", "Converted Date"]].head(10)

,Date,Cleaned Date,Converted Date
0,24 Feb 2024,24 Feb 2024,2024-02-24
1,24 Feb 2024,24 Feb 2024,2024-02-24
3,24 Feb 2024,24 Feb 2024,2024-02-24
4,24 Feb 2024,24 Feb 2024,2024-02-24
7,23 Feb 2024,23 Feb 2024,2024-02-23
9,23 Feb 2024,23 Feb 2024,2024-02-23
10,23 Feb 2024,23 Feb 2024,2024-02-23
11,23 Feb 2024,23 Feb 2024,2024-02-23
12,22 Feb 2024,22 Feb 2024,2024-02-22
13,22 Feb 2024,22 Feb 2024,2024-02-22


In [577]:
activity_df.dropna(subset = ["Converted Date"], inplace = True)

In [578]:
activity_df.drop(columns = ["Date", "Cleaned Date", "Time"], inplace = True)

In [579]:
# Current order of columns
current_columns = activity_df.columns.tolist()

# Desired order: moving "Converted Date" to be the second column
new_order = current_columns[:2] + ["Converted Date"] + current_columns[2:-1]

# Reorder the columns in the dfFrame
activity_df = activity_df[new_order]

# Display the first few rows of the reordered dfframe
activity_df.head()
activity_df.rename(columns={"Converted Date": "Date"}, inplace=True)
activity_df["Date"] = pd.to_datetime(activity_df["Date"])
activity_df.drop(columns = ["Product", "Details", "Status"], axis = 1, inplace = True)

In [580]:
activity_df.head(20)

,Paid Info,Date
0,Paid 492.00 using Bank Account XXXXXX5367,2024-02-24
1,Paid 477.22 to Zomato using Bank Account XXXXX...,2024-02-24
3,Paid 644.00 using Bank Account XXXXXX5367,2024-02-24
4,Paid 140.00 to HOTEL SRI RAGHAVENDRA using Ban...,2024-02-24
7,Paid 40.00 to MUPPARAPU LAVAN using Bank Accou...,2024-02-23
9,Paid 3572.50 to BOOKMYSHOW using Bank Account ...,2024-02-23
10,Paid 20.00 to Mr S RAMESH using Bank Account X...,2024-02-23
11,Paid 100.00 to Mr S RAMESH using Bank Account ...,2024-02-23
12,Paid 40.00 to VENKATESH K using Bank Account X...,2024-02-22
13,Paid 815.00 using Bank Account XXXXXX5367,2024-02-22


In [581]:
# activity_df.to_csv("Activity_df.csv", index = False)

In [582]:
activity_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 265 entries, 0 to 562
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Paid Info  265 non-null    object        
 1   Date       265 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 6.2+ KB


In [583]:
activity_df.head(20)

,Paid Info,Date
0,Paid 492.00 using Bank Account XXXXXX5367,2024-02-24
1,Paid 477.22 to Zomato using Bank Account XXXXX...,2024-02-24
3,Paid 644.00 using Bank Account XXXXXX5367,2024-02-24
4,Paid 140.00 to HOTEL SRI RAGHAVENDRA using Ban...,2024-02-24
7,Paid 40.00 to MUPPARAPU LAVAN using Bank Accou...,2024-02-23
9,Paid 3572.50 to BOOKMYSHOW using Bank Account ...,2024-02-23
10,Paid 20.00 to Mr S RAMESH using Bank Account X...,2024-02-23
11,Paid 100.00 to Mr S RAMESH using Bank Account ...,2024-02-23
12,Paid 40.00 to VENKATESH K using Bank Account X...,2024-02-22
13,Paid 815.00 using Bank Account XXXXXX5367,2024-02-22


In [584]:
# activity_df.to_csv("New_Activity.csv", index = False)

### Last 10 days data

In [585]:
# Get the last date in the dfframe
last_date = activity_df["Date"].max()

# Filter the df for the last 10 days
last_10_days_df = activity_df[(activity_df["Date"] > (last_date - pd.Timedelta(days = 10))) &
                                (activity_df["Date"] <= last_date)]

last_10_days_df.reset_index(drop=True, inplace=True)
last_10_days_df

,Paid Info,Date
0,Paid 492.00 using Bank Account XXXXXX5367,2024-02-24
1,Paid 477.22 to Zomato using Bank Account XXXXX...,2024-02-24
2,Paid 644.00 using Bank Account XXXXXX5367,2024-02-24
3,Paid 140.00 to HOTEL SRI RAGHAVENDRA using Ban...,2024-02-24
4,Paid 40.00 to MUPPARAPU LAVAN using Bank Accou...,2024-02-23
5,Paid 3572.50 to BOOKMYSHOW using Bank Account ...,2024-02-23
6,Paid 20.00 to Mr S RAMESH using Bank Account X...,2024-02-23
7,Paid 100.00 to Mr S RAMESH using Bank Account ...,2024-02-23
8,Paid 40.00 to VENKATESH K using Bank Account X...,2024-02-22
9,Paid 815.00 using Bank Account XXXXXX5367,2024-02-22


In [586]:
# Get unique transactions in the Name / Description column
unique_transactions = activity_df["Paid Info"].unique()
len(unique_transactions)

199

In [587]:
RESPONSE_JSON = {
    "Service Name": {
        "Category": "Category name",
        "Date": "Date of Transaction",
        "Cost": "Transaction amount",
    },
    "Service Name": {
        "Category": "Category name",
        "Date": "Date of Transaction",
        "Cost": "Transaction amount",
    },
    "Service Name": {
        "Category": "Category name",
        "Date": "Date of Transaction",
        "Cost": "Transaction amount",
    },
    "Service Name": {
        "Category": "Category name",
        "Date": "Date of Transaction",
        "Cost": "Transaction amount",
    },
    "Service Name": {
        "Category": "Category name",
        "Date": "Date of Transaction",
        "Cost": "Transaction amount",
    },
}

In [588]:
TEMPLATE = """
Paid_Info: {text}
Date: {date}
Number of Transactions: {Total_Transactions}
You are expert Finance Analyzer.
There are {Total_Transactions} number of transactions with dates of transaction.
Categorize each and every transaction. 
Note: Create a minimum of 6 categories.
Make sure to format the response like response_json below and use it as a guide.
{response_json}
"""

In [589]:
Paid_Info = last_10_days_df["Paid Info"].to_list()
Date = last_10_days_df["Date"].to_list()

In [590]:
prompt = PromptTemplate(
    input_variables = ["text", "date", "Total_Transactions", "response_json"], 
    template = TEMPLATE
)

In [591]:
chain = LLMChain(llm = llm, prompt = prompt, verbose = True)

In [592]:
# How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response = chain(
        {
            "text": Paid_Info,
            'date': Date,
            "Total_Transactions": len(last_10_days_df),
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new LLMChain chain...
Prompt after formatting:

Paid_Info: ['Paid 492.00 using Bank Account XXXXXX5367', 'Paid 477.22 to Zomato using Bank Account XXXXXX5367', 'Paid 644.00 using Bank Account XXXXXX5367', 'Paid 140.00 to HOTEL SRI RAGHAVENDRA using Bank Account XXXXXX5367', 'Paid 40.00 to MUPPARAPU LAVAN using Bank Account XXXXXX5367', 'Paid 3572.50 to BOOKMYSHOW using Bank Account XXXXXX5367', 'Paid 20.00 to Mr S RAMESH using Bank Account XXXXXX5367', 'Paid 100.00 to Mr S RAMESH using Bank Account XXXXXX5367', 'Paid 40.00 to VENKATESH K using Bank Account XXXXXX5367', 'Paid 815.00 using Bank Account XXXXXX5367', 'Paid 15.00 to AMRIT ENTERPRISES using Bank Account XXXXXX5367', 'Paid 70.00 to VENKATESH JANGI using Bank Account XXXXXX5367', 'Paid 499.00 to Netflix Entertainment Services India LLP using Bank Account XXXXXX5367', 'Paid 549.00 to UDEMY INDIA LLP COLLECTIONS using Bank Account XXXXXX5367', 'Paid 20.00 to MUPPARAPU LAVAN using Bank Account XXXXXX5367', 'Paid 15.0

In [593]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 1568
Prompt Tokens: 1282
Completion Tokens: 286
Total Cost: 0.002495


In [594]:
response

{'text': '{\n    "Zomato": {\n        "Category": "Food & Dining",\n        "Date": "2024-02-24",\n        "Cost": "477.22"\n    },\n    "HOTEL SRI RAGHAVENDRA": {\n        "Category": "Accommodation",\n        "Date": "2024-02-24",\n        "Cost": "140.00"\n    },\n    "BOOKMYSHOW": {\n        "Category": "Entertainment",\n        "Date": "2024-02-23",\n        "Cost": "3572.50"\n    },\n    "Netflix Entertainment Services India LLP": {\n        "Category": "Entertainment",\n        "Date": "2024-02-22",\n        "Cost": "499.00"\n    },\n    "UDEMY INDIA LLP COLLECTIONS": {\n        "Category": "Education",\n        "Date": "2024-02-22",\n        "Cost": "549.00"\n    },\n    "StockEdge": {\n        "Category": "Investment",\n        "Date": "2024-02-21",\n        "Cost": "149.90"\n    },\n    "APOLLO PHARMACY": {\n        "Category": "Healthcare",\n        "Date": "2024-02-15",\n        "Cost": "323.20"\n    }\n}',
 'date': [Timestamp('2024-02-24 00:00:00'),
  Timestamp('2024-02-24

In [595]:
response.get("text")

'{\n    "Zomato": {\n        "Category": "Food & Dining",\n        "Date": "2024-02-24",\n        "Cost": "477.22"\n    },\n    "HOTEL SRI RAGHAVENDRA": {\n        "Category": "Accommodation",\n        "Date": "2024-02-24",\n        "Cost": "140.00"\n    },\n    "BOOKMYSHOW": {\n        "Category": "Entertainment",\n        "Date": "2024-02-23",\n        "Cost": "3572.50"\n    },\n    "Netflix Entertainment Services India LLP": {\n        "Category": "Entertainment",\n        "Date": "2024-02-22",\n        "Cost": "499.00"\n    },\n    "UDEMY INDIA LLP COLLECTIONS": {\n        "Category": "Education",\n        "Date": "2024-02-22",\n        "Cost": "549.00"\n    },\n    "StockEdge": {\n        "Category": "Investment",\n        "Date": "2024-02-21",\n        "Cost": "149.90"\n    },\n    "APOLLO PHARMACY": {\n        "Category": "Healthcare",\n        "Date": "2024-02-15",\n        "Cost": "323.20"\n    }\n}'

In [596]:
ok  = response.get("text")
ok = json.loads(ok)
ok

{'Zomato': {'Category': 'Food & Dining',
  'Date': '2024-02-24',
  'Cost': '477.22'},
 'HOTEL SRI RAGHAVENDRA': {'Category': 'Accommodation',
  'Date': '2024-02-24',
  'Cost': '140.00'},
 'BOOKMYSHOW': {'Category': 'Entertainment',
  'Date': '2024-02-23',
  'Cost': '3572.50'},
 'Netflix Entertainment Services India LLP': {'Category': 'Entertainment',
  'Date': '2024-02-22',
  'Cost': '499.00'},
 'UDEMY INDIA LLP COLLECTIONS': {'Category': 'Education',
  'Date': '2024-02-22',
  'Cost': '549.00'},
 'StockEdge': {'Category': 'Investment',
  'Date': '2024-02-21',
  'Cost': '149.90'},
 'APOLLO PHARMACY': {'Category': 'Healthcare',
  'Date': '2024-02-15',
  'Cost': '323.20'}}

In [597]:
df = pd.DataFrame(ok)
df = df.T
df

,Category,Date,Cost
Zomato,Food & Dining,2024-02-24,477.22
HOTEL SRI RAGHAVENDRA,Accommodation,2024-02-24,140.00
BOOKMYSHOW,Entertainment,2024-02-23,3572.50
Netflix Entertainment Services India LLP,Entertainment,2024-02-22,499.00
UDEMY INDIA LLP COLLECTIONS,Education,2024-02-22,549.00
StockEdge,Investment,2024-02-21,149.90
APOLLO PHARMACY,Healthcare,2024-02-15,323.20


In [598]:
# Group and sum the costs by category
category_sum = df.groupby('Category')['Cost'].sum().reset_index()

# Create the pie chart
fig = px.pie(category_sum, names='Category', values='Cost', title='Expense Distribution by Category')

# Show the figure
fig.show()

In [599]:
df['Date'] = pd.to_datetime(df['Date']).dt.date
df['Cost'] = pd.to_numeric(df['Cost'], errors='coerce')

# Summing the costs for each day
daily_total_cost = df.groupby('Date')['Cost'].sum().reset_index()

# Creating the bar plot with the cleaned df
daily_total_cost_cleaned_plot = px.bar(daily_total_cost, x='Date', y='Cost', 
                                    title='Total Daily Expenses',
                                    labels={'Cost': 'Total Cost'})


# Ensure the Date column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])


# Scatter Plot - Costs by Date with Category as Color
scatter_plot = px.scatter(df, x='Date', y='Cost', color='Category', 
                        title='Scatter Plot of Costs by Date')


# Pie Chart - Distribution by Category
pie_chart = px.pie(df, names='Category', title='Expense Distribution by Category')

# Box Plot - Cost Distribution by Category
box_plot = px.box(df, x='Category', y='Cost', title='Cost Distribution by Category')

# Create Panel Dashboard
pn.extension('plotly')
dashboard = pn.Tabs(
    ('Daily Expenses', daily_total_cost_cleaned_plot),
    ('Scatter Plot', scatter_plot),
    ('Pie Chart', pie_chart),
    ('Box Plot', box_plot)
)

# Serve the dashboard
dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'9c4a800c-e733-47d6-a837-f8beabcb4ed0': {'version…